# Tutorial 1: Discrete models from GMSH and BCs

In this **tutorial**, we will learn

   -  How to load a discrete model from a GMSH file
   -  How to impose Dirichlet and Neumann boundary conditions

## Problem statement

We want to solve the **Poisson equation** on the 3D domain depicted in the next figure with Dirichlet, Robin, and homogeneous Neumann boundary conditions. Dirichlet boundary conditions are applied on $\Gamma_{\rm D}$, being the outer sides of the prism (marked in red). Robin conditions are applied to the internal boundaries $\Gamma_{\rm G}$, $\Gamma_{\rm Y}$, and $\Gamma_{\rm B}$ (marked in green, yellow and blue respectively). Robin boundary conditions are expressed as a convective heat loss (Newton's law of cooling). Finally, homogeneous Neumann boundary conditions are applied in $\Gamma_{\rm W}$, the remaining portion of the boundary (marked in white).

![](model-r1-2.png)

 Formally, the problem to solve is: find the scalar field $u$ such that

$$
\left\lbrace
\begin{aligned}
-\Delta u &= f  \ &\text{in} \ \Omega,\\
u &= g \ &\text{on}\ \Gamma_{\rm D},\\
- \nabla u\cdot \boldsymbol{n} &= h (u - u_{\rm o}) \ &\text{on}\  \Gamma_{\rm R},\\
- \nabla u\cdot \boldsymbol{n} &= 0 \ &\text{on}\  \Gamma_{\rm N},\\
\end{aligned}
\right.
$$
 being $\boldsymbol{n}$ the outwards unit normal vector to the Robin and Neumann boundary, $\Gamma_{\rm R} \doteq \Gamma_{\rm G}\cup \Gamma_{\rm Y} \cup \Gamma_{\rm B}$ and $\Gamma_{\rm N} \doteq \Gamma_{\rm W}$, respectively. In this example, we chose $f(x) = 1$, $g(x) = 2$, $h(x)=3$ and $u_{\rm o}(x) = 1$. The variable $x$ is the position vector $x=(x_1,x_2,x_3)$.

## Numerical scheme

To solve this PDE, we use a conventional Galerkin finite element (FE) method with conforming Lagrangian FE spaces (see, e.g., [1] for specific details on this formulation). The weak form associated with this formulation is: find $u\in U_g$ such that $ a(u,v) = b(v) $ for all $v\in V_0$, where $U_g$ and $V_0$ are the subset of functions in $H^1(\Omega)$ that fulfill the Dirichlet boundary condition $g$ and $0$ respectively. The bilinear and linear forms for this problems are
$$
  a(u,v) \doteq \int_{\Omega} \nabla u \cdot \nabla v \ {\rm d}\Omega + \int_{\Gamma_{\rm R}} h u \ v \ {\rm d}\Gamma_{\rm R}, \quad b(v) \doteq \int_{\Omega} f\ v  \ {\rm  d}\Omega + \int_{\Gamma_{\rm R}} h u_{\rm o} \ v \ {\rm d}\Gamma_{\rm R},
$$
where the homogeneous Neumann condition on $\Gamma_{\rm N}$ has already been imposed.

The problem is solved numerically by approximating the spaces $U_g$ and $V_0$ by their discrete counterparts associated with a FE mesh of the computational domain $\Omega$. As we have anticipated, we consider standard conforming Lagrangian FE spaces for this purpose.

**In this tutorial, we focus on building discrete models from files generated by GMSH and setting up the boundary conditions in Gridap.** As usual, the first step is to load the Gridap library in the code.

In [ ]:
using Gridap
using GridapGmsh

## Loading and visualising discrete models from GMSH files

 In the following line, we build an instance of `DiscreteModel` by loading a `json` file.

In [ ]:
msh_file = joinpath(@__DIR__,"toy_model.msh")
println(msh_file)
model = GmshDiscreteModel(msh_file)

The file `"model.json"` is a regular `json` file that includes a set of fields that describe the discrete model. It was generated by using together the [GMSH](http://gmsh.info/) mesh generator and the [GridapGmsh](https://github.com/gridap/GridapGmsh.jl) package. First, we generate a `"model.msh"` file with GMSH (which contains a FE mesh and information about user-defined physical boundaries in {GMSH} format). Then, this file is converted to the Gridap-compatible `"model.json"` file using the conversion tools available in the GridapGmsh package. See the documentation of the [GridapGmsh](https://github.com/gridap/GridapGmsh.jl) for more information.

You can easily inspect the generated discrete model in [Paraview](https://www.paraview.org/) by writing it in `vtk` format.

In [ ]:
writevtk(model,"model")

The previous line generates four different files `model_0.vtu`, `model_1.vtu`, `model_2.vtu`, and `model_3.vtu` containing the vertices, edges, faces, and cells present in the discrete model. Moreover, you can easily inspect which boundaries are defined within the model.

For instance, if you want to see which faces of the model are on the boundary $\Gamma_{\rm B}$ (i.e., the walls of the circular perforation), open the file `model_2.vtu` and chose coloring by the element field "circle". You should see that only the faces on the circular hole have a value different from zero (see next figure).

![](fig_faces_on_circle.png)

It is also possible to see which vertices are on the Dirichlet boundary $\Gamma_{\rm D}$. To do so, open the file `model_0.vtu` and chose coloring by the field "sides" (see next figure).

![](fig_vertices_on_sides.png)

That is, the boundary $\Gamma_{\rm B}$ (i.e., the walls of the circular hole) is called "circle" and the Dirichlet boundary $\Gamma_{\rm D}$ is called "sides" in the model. In addition, the walls of the triangular hole $\Gamma_{\rm G}$ and the walls of the square hole $\Gamma_{\rm Y}$ are identified in the model with the names "triangle" and "square" respectively. You can easily check this by opening the corresponding file in Paraview.

## Setting up (strong) Dirichlet BCs during FE space creation

Next, we generate the discrete approximation of the finite element spaces $V_0$ and $U_g$ (i.e. the test and trial FE spaces) of the problem. To do so, first, we are going to build a discretization of $V_0$ as the standard Conforming Lagrangian FE space (with zero boundary conditions) associated with the discretization of the computational domain. The approximation of the FE space $V_0$ is built as follows:

In [ ]:
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V0    = TestFESpace(model,reffe,dirichlet_tags="sides")

Here, we have used the `TestFESpace` constructor, which constructs a particular FE space (to be used as a test space) from a set of options described as positional and key-word arguments. We pass the identifiers of the Dirichlet boundary via the `dirichlet_tags` argument. In this case, we mark as Dirichlet all objects of the discrete model identified with the `"sides"` tag. Since this is a test space, the corresponding shape functions vanishes at the Dirichlet boundary.

Once the space $V_0$ is discretized in the code, we proceed with the approximation of the trial space $U_g$.

In [ ]:
g(x) = 2.0
Ug = TrialFESpace(V0,g)

To this end, we have used the `TrialFESpace` constructors. Note that we have passed a function representing the value of the Dirichlet boundary condition, when building the trial space.

## Numerical integration of weak Robin (or non-homogeneous Neumann) BCs

The next step is to set up the machinery to perform the integrals in the weak form numerically. Here, we need to compute integrals on the interior of the domain $\Omega$ and on the Robin boundary $\Gamma_{\rm R}$.

To integrate the Robin integral terms in the weak form, we need a special type of triangulation, represented by the type `BoundaryTriangulation`. Essentially, a `BoundaryTriangulation` is a particular type of `Triangulation` that is aware of which cells in the model are touched by faces on the boundary. We build an instance of this type from the discrete model and the names used to identify the Robin boundary as follows:

In [ ]:
robintags = ["circle", "triangle", "square"]
Γ = Boundary(model,tags=robintags)

In addition, we associate a Lebesgue measure with a quadrature of degree 2 to the cells in the triangulation for the Robin boundary $\Gamma$.

In [ ]:
degree = 2
dΓ = Measure(Γ,degree)

For integrating on the domain $\Omega$, we build the following triangulation and the corresponding Lebesgue measure.

In [ ]:
Ω = Interior(model)
dΩ = Measure(Ω,degree)

## Writing the weak form

With all the ingredients presented so far, we are ready to define the weak form. This is done by defining to functions representing the bi-linear and linear forms:

In [ ]:
f(x)  = 1.0
h(x)  = 3.0
uₒ(x) = 1.0

a(u,v) = ∫( ∇(u)⋅∇(v) )dΩ + ∫( h*u*v )dΓ
b(v)   = ∫( f*v )dΩ + ∫( h*uₒ*v )dΓ

Note that, in order to integrate $b$ as defined above, we need to overload the product rule for the pair (h,uₒ) as follows.

In [ ]:
import Base.*
*(::typeof(h),::typeof(uₒ)) = x -> h(x)*uₒ(x)

## Solving the problem and visualising the solution

Finally, we can build and solve the FE problem

In [ ]:
op = AffineFEOperator(a,b,Ug,V0)
uh = solve(op)

and inspect the solution by writing it into a `vtk` file:

In [ ]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

 which will generate a file named `results.vtu` having a nodal field named `"uh"` containing the solution of our problem (see next figure).

![](fig_uh.png)


    Note that set up and numerical integration of non-homogeneous Neumann conditions would be a particular case of the Robin one (with only the integral term on the right-hand side linear form $b$).

## References

This tutorial has been adapted from the [Gridap Tutorial 1: Poisson equation](https://gridap.github.io/Tutorials/dev/pages/t001_poisson/)

**Tutorial done!**

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*